# Testing Drone

## Logging basic info

In [8]:
group_number = 16

In [6]:
# This is an example from the Crazyflie Python API.
# See https://github.com/bitcraze/crazyflie-lib-python/blob/master/examples/basiclogSync.py

import logging
import time

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger

# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)


# Initialize the low-level drivers (don't list the debug drivers)
cflib.crtp.init_drivers(enable_debug_driver=False)
# Scan for Crazyflies and use the first one found
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    lg_stab = LogConfig(name='Stabilizer', period_in_ms=10)
    lg_stab.add_variable('stabilizer.roll', 'float')
    lg_stab.add_variable('stabilizer.pitch', 'float')
    lg_stab.add_variable('stabilizer.yaw', 'float')

    cf = Crazyflie(rw_cache='./cache')
    with SyncCrazyflie(available[0][0], cf=cf) as scf:
        with SyncLogger(scf, lg_stab) as logger:
            endTime = time.time() + 10

            for log_entry in logger:
                timestamp = log_entry[0]
                data = log_entry[1]
                logconf_name = log_entry[2]

                print('[%d][%s]: %s' % (timestamp, logconf_name, data))

                if time.time() > endTime:
                    break


Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/16/2M
radio://0/16/2M
radio://0/16/2M
[18353][<cflib.crazyflie.log.LogConfig object at 0x0000023FECAA31C0>]: {'stabilizer.roll': 0.5201911926269531, 'stabilizer.pitch': -0.4800904095172882, 'stabilizer.yaw': 2.8758347034454346}
[18363][<cflib.crazyflie.log.LogConfig object at 0x0000023FECAA31C0>]: {'stabilizer.roll': 0.516617476940155, 'stabilizer.pitch': -0.5006234645843506, 'stabilizer.yaw': 2.882582664489746}
[18373][<cflib.crazyflie.log.LogConfig object at 0x0000023FECAA31C0>]: {'stabilizer.roll': 0.5144556760787964, 'stabilizer.pitch': -0.5154309272766113, 'stabilizer.yaw': 2.887367010116577}
[18383][<cflib.crazyflie.log.LogConfig object at 0x0000023FECAA31C0>]: {'stabilizer.roll': 0.5098692178726196, 'stabilizer.pitch': -0.5304557681083679, 'stabilizer.yaw': 2.8932816982269287}
[18393][<cflib.crazyflie.log.LogConfig object at 0x0000023FECAA31C0>]: {'stabilizer.roll': 0.5701419115066528, 'stabilizer.pitch': -0.73104

## Simple PID test

In [7]:
import logging
import random
import time
import numpy as np

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncLogger import SyncLogger

URI = f'radio://0/{group_number}/2M'

# Only output errors from the logging framework
logging.basicConfig(level=logging.ERROR)

log_keys = ['e_x',
            'e_y',
            'e_roll',
            'e_pitch',
            'u2_pwm',
            'u3_pwm',]

other_log_keys = []

if __name__ == '__main__':
    cflib.crtp.init_drivers(enable_debug_driver=False)

    with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:
        cf = scf.cf

        
        print('Initializing as PID')
        cf.param.set_value('stabilizer.controller', '1')

    
        print('Taking off!')

        for y in range(10):
            cf.commander.send_hover_setpoint(0, 0, 0, y / 25)
            #cf.commander.send_stop_setpoint()
            time.sleep(0.1)

        for _ in range(50):
            cf.commander.send_hover_setpoint(0, 0, 0, 0.5)
            #cf.commander.send_stop_setpoint()
            time.sleep(0.1)

        for y in range(10):
            cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25)
            time.sleep(0.1)

        for i in range(10):
            cf.commander.send_stop_setpoint()
            time.sleep(0.1)

print('Done!') 

Initializing as PID
Taking off!
Done!
